# Create media designs using ART

We use ART to provide suggested designs for media components for which to get phenotypic data.  We will use absorbance at 340nm as assay for this experiment.

For DBTL 8 we create X designs from an exploratory mode with $\alpha=1.$ and relative recommendations distance X%, X designs from the exploitation mode ($\alpha=0$) and relative recommendations distance X%, and 1 design being close to the standard media as a control, totalling 16 designs in triplicates.


Tested using **ART Prod** kernel on skynet.ese.lbl.gov

## Inputs and output

**Required file to run this notebook:**
- `Putida_media_bounds_extended.csv`
- `standard_recipe_concentrations_extended.csv`
- EDD study slug(s)

**File generated by running this notebook**
- 
-
-

The files are stored in the user defined directory. 

## Setup

Clone the git repository with the `ART` library 

`git clone https://github.com/JBEI/AutomatedRecommendationTool.git`  
<!-- <font color='red'> _____ -->
<!-- **WE SHOULD TALK ABOUT LICENSING HERE!!!** </font> -->

or pull the latest version. 

Information about licensing ART is available at https://github.com/JBEI/ART.

Importing needed libraries:

In [ ]:
import os
import sys
import re
import warnings
import pickle
 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path




Import ART

In [ ]:
sys.path.append('../')        # Make sure this is the location for the ART library 
sys.path.append('../media_compiler')

from art.core import RecommendationEngine 
import art.plot as plot
import art.utility as utils
import edd_utils as eddu
from core import designs_pairwise
import warning_utils
warning_utils.filter_end_user_warnings()



## User parameters

In [ ]:
CYCLE = 4

user_params = {
    'bounds_file': f'../flaviolin yield data/Putida_media_bounds_yield.csv',
    'output_dir': f'../flaviolin yield data/DBTL{CYCLE}', # Folder for output files,
    'standard_media_file': '../flaviolin yield data/standard_recipe_concentrations.csv', #location of the standard media recipe
    'study_slug_1': 'flav_c3_dbtl1', #DBTL1
    'study_slug_2': 'flav_c3_dbtl2',#DBTL2
    'study_slug_3': 'flav_c3_dbtl3',#DBTL3
    'edd_server': 'public-edd.jbei.org',
    'username': '<YOURUSERNAMEHERE>',
}

In [ ]:
os.makedirs(user_params['output_dir'], exist_ok=True) #make the directory to store the output files

### Defining media components and the number of instances (designs) to be created

Specify which components to explore:

In [ ]:
user_params['components'] = [
    'H3BO3[mM]',
    'Glucose[mM]',
    'K2SO4[mM]',
    'K2HPO4[mM]',
    'FeSO4[mM]',
    'NH4Cl[mM]',
    'MgCl2[mM]',
    'NaCl[mM]',
    '(NH4)6Mo7O24[mM]',
    'CoCl2[mM]',
    'CuSO4[mM]',
    'MnSO4[mM]',
    'ZnSO4[mM]',
    # 'CaCl2[mM]'    
]

user_params['response'] = 'OD340yield' #this will be the parameter we will maximize

Here we specify how many instances (designs) we want to create and how many replicates:

In [ ]:
user_params['n_instances_explor'] = 7 # number of exploration recommendations
user_params['n_instances_exploit'] = 8 # number of exploitation recommendations
user_params['n_replicates'] = 3 # number of replicates generated in the biolector plate.
user_params['seed'] = 42
#The biolector plate we are using here is a 48-well plate. For 3 replicates, the total number of conditions tested will need to be 48/3 = 16. In this case 13 exploitation, 2 exploration and 1 standard. 

Lastly we specify the exploration and exploitation parameters (alpha) and the number of iterations that Parallel tempering will go through

In [ ]:
user_params['alpha_explore'] = 1
user_params['alpha_exploit'] = 0 
user_params['n_iter'] = 1e5

In [ ]:
user_params['scale_input_vars'] = False
user_params['cross_val'] = True
user_params['recommend'] = False

## Load the data

In [ ]:
df_stand = pd.read_csv(user_params['standard_media_file']).set_index("Component")

In [ ]:
study_slug_1 = user_params['study_slug_1']
study_slug_2 = user_params['study_slug_2']
study_slug_3 = user_params['study_slug_3']


edd_server = user_params['edd_server']
username = user_params['username']

In [ ]:
try:
    session = eddu.login(edd_server=edd_server, user=username)
except:
    print('ERROR! Connection to EDD failed. We will try to load data from disk...')
    load_from_disk = True
else:
    print('OK! Connection to EDD successful. We will try to load data from EDD...')    
    load_from_disk = False

In [ ]:
try:
    df_1 = eddu.export_study(session, study_slug_1, edd_server=edd_server)
    df_2 = eddu.export_study(session, study_slug_2, edd_server=edd_server)
    df_3 = eddu.export_study(session, study_slug_3, edd_server=edd_server)
    load_from_disk = False
except (NameError, AttributeError, KeyError):
    print(f'ERROR! Not able to export the study.')
    load_from_disk = True

In [ ]:

if load_from_disk:
    df_1 = pd.read_csv('../flaviolin yield data/df_dbtl1-yield.csv')
    df_2 = pd.read_csv('../flaviolin yield data/df_dbtl2-yield.csv')
    df_3 = pd.read_csv('../flaviolin yield data/df_dbtl3-yield.csv')
else:
    df_1.to_csv('../flaviolin yield data/df_dbtl1-yield.csv')
    df_2.to_csv('../flaviolin yield data/df_dbtl2-yield.csv')



## Preprocess the data

Concatenate the six studies:

In [ ]:
df = pd.concat([df_1, df_2, df_3])

In [ ]:
df.head()

### match the formatting between DBTL1-5 to the formatting of DBTL6


In DBTL1-6 and DBTL7+ the columns "Protocol" and "MEasurement Type" are inverted. Correct that by inverting their names in DBTL1-6

In [ ]:
#df[['Protocol', 'Measurement Type']] = df[['Measurement Type', 'Protocol']]

In [ ]:
#df = pd.concat([df, df_6])

### clean up the df

In [ ]:
df.head()

In [ ]:
df.tail()

Drop unnecessary columns:

In [ ]:
df = df.loc[:,['Line Name','Line Description','Measurement Type','Value']]


Pivot the dataframe: This will generate a line for each sample with the corresponding OD600 and OD340 values

In [ ]:
df = df.pivot(index=["Line Name", "Line Description"], columns="Measurement Type", values="Value")
df.reset_index(inplace=True)


Drop OD600 since we are not using it

### Adding media information to the data frame

Add columns for each component:

In [ ]:
components = re.split(': |, ', df['Line Description'][0])[::2]
for comp in components:
    df[comp] = None
# df['CaCl2[mM]'] = 0 # add a column for CaCl2. 
#CaCl2 is only added to DBTL 7 and 8 so it doesn't exist in the line descriptions for DBTL 1-6. 
#Concentration of CaCl2 will be 0 for DBTL 1-6

#df['Kan[g/L]'] = None # add a column for Kan. 
#Kan concentration is 0.05 g/l. 
#This is the same concentration for all cases and is not used by ART.
#this is in the Line descriptions only for DBTL8



For every component in the line description assign the right value in each column created previously.

In [ ]:
for i in range(len(df)):
    values = re.split(': |, ', df['Line Description'][i])[1::2]
    for c, value in enumerate(values):
        df.iloc[i, (3+c)] = float(value)


df.drop(columns='Line Description', inplace=True)


### Include Yield and clean up the training data

Include the Yield column:

In [ ]:
df['OD340yield'] = df['OD340']/df['Glucose[mM]']
df['OD340yield']

In [ ]:
# df.iloc[df['OD340'].idxmax(),:]
# df['OD340'].idxmax()
# df = df[df['OD340'] < .86]

# outliers = ['C2.1_WA7_B1-R1','C2.1_WB8_C2-R1']

# df = df[~df['Line Name'].isin(outliers)]

In [ ]:
df.head()

In [ ]:
df.tail()

Remove all the low performing strains

bar plot the training data

Define the control lines. In DBTL 1 and 2, controls were wells F5 to F8. In other DBTL cycles the control is stored in the last column (D8, E8, F8).

In [ ]:
control_lines = df[df['Line Name'].str.find('WF5_F8') > 0]

#control_lines = control_lines.append(df[df['Line Name'].str.find('WD8_F8') > 0])

control_lines = pd.concat([control_lines, df[df['Line Name'].str.find('WD8_F8') > 0]], ignore_index=True)

### Convert the data to EDD format

Pivot the dataframe back to EDD format, including all the components names and protocols:

In [ ]:
df_stacked = df.set_index('Line Name').stack().reset_index()
df_stacked.columns = ['Line Name', 'Measurement Type', 'Value']
df_stacked

In [ ]:
df_stacked = df.set_index('Line Name').stack().reset_index()
df_stacked.columns = ['Line Name', 'Measurement Type', 'Value']

### OD340 of the highest performing strains is at 0.7. 
### I multiply the data by 1000 to bring it up to ~700 to improve parallel tempering performance
# df_stacked.loc[df_stacked["Measurement Type"] == "OD340yield", 'Value'] = df_stacked.loc[df_stacked["Measurement Type"] == "OD340yield", 'Value'] 

df_stacked


## Define functions we will use to train art, generate recommendations and plot the results

In [ ]:
def hist_recommendations(df_rec, hist_components):
    n_plots = len(hist_components)
    n_cols = 3
    n_rows = (n_plots + 1) // n_cols    
    fig, axs = plt.subplots(n_rows,n_cols, figsize = (7,15),facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace = 0.2, wspace=0.3)
    axs = axs.ravel()
    #axs.update(wspace=0.5, hspace=0.5)
    for iplot in range(n_plots):
        v = hist_components[iplot]
        axs[iplot].hist(df_rec[v])
        axs[iplot].set_title(v)
    

In [ ]:
def plot_train_recs(df_train, df_rec, train_label, rec_label):
    names_train = df_train['Line Name']
    y_train = df_train[train_label]

    names_rec = [f'rec{i}' for i in range(len(df_target))]
    y_rec = df_target[rec_label]
    fig, ax = plt.subplots()
    ax.bar(names_train, y_train, color='blue')
    ax.bar(names_rec, y_rec, color='red')

In [ ]:
def train_art(df, user_params,art_filename):
    art_params = {
    'input_vars': user_params['components'],
    'response_vars': [user_params['response']],
    'bounds': utils.read_table(user_params['bounds_file']), # file with bounds# input variables, i.e. features
    'scale_input_vars': user_params['scale_input_vars'],
    'cross_val': user_params['cross_val'],
    'recommend': user_params['recommend'],
    'output_dir': user_params['output_dir'],  # directory to store this output
    'verbose': 1,
    'num_tpot_models': 4,
    'max_mcmc_cores': 4,
    'seed': user_params['seed'],
    'build_model' : True,
    'recommend' : False
    }
    
    # art_params['bounds'] = art_params['bounds'].drop(index=[0,1,3])
    
    art = RecommendationEngine(df=df, **art_params)
    
    art.save()
    
    orig_file_name = f"{art_params['output_dir']}/art.cpkl"
    new_file_name = f"{art_params['output_dir']}/{art_filename}"
    os.rename(orig_file_name, new_file_name)

    return art

In [ ]:
def generate_recommendations(art, user_params, draws_filename, alpha, n_recs, rel_rec_distance):
    art.niter = user_params['n_iter']
    #user_params['aplha_exploit'] = 0 
    art.num_recommendations = n_recs # 
    art.rel_rec_distance = rel_rec_distance # Default is 0.2
    art.alpha = alpha 
    draws = art.parallel_tempering_opt()
    art.recommend(draws)
    orig_file_name = f"{user_params['output_dir']}/draws.txt"
    new_file_name = f"{user_params['output_dir']}/{draws_filename}"
    os.rename(orig_file_name, new_file_name)
    return art, draws

In [ ]:
def process_and_plot_recommendations(art):
    df_rec = art.recommendations
    df_rec.rename(columns={'OD340': 'OD340_pred'}, inplace=True)

    predicted_mean, predicted_std = art.post_pred_stats(
                df_rec.values[:, :-1]
            ) # posterior predictive statistics: mean and std

    df_rec['OD340_std'] = predicted_std

    df_rec['Label'] = 'exploitation'
    return df_rec

In [ ]:
def load_art_get_train_recs(path_to_art):
    with open(path_to_art, "rb") as input_file:
        art = pickle.load(input_file)
    df_train = pd.DataFrame(art.X, columns = art.input_vars)
    df_train[art.response_vars[0]] = art.y
    
    df_rec = art.recommendations
    predicted_mean, predicted_std = art.post_pred_stats(
                df_rec.values[:, :-1]
            ) # posterior predictive statistics: mean and std
    df_rec[art.response_vars[0]+'_pred'] = predicted_mean
    df_rec[art.response_vars[0]+'_pred'] = predicted_std
    return df_train, df_rec
    

In [ ]:
def plot_dfs(dfs, cols, labels, xlabel, ylabel, title):
    
    fig, ax = plt.subplots()
    counter = 0
    for i in range(len(dfs)):
        ax.scatter(np.array(range(len(dfs[i][cols[i]])))+counter, dfs[i][cols[i]], label = labels[i])
        counter+=len(dfs[i][cols[i]])
        # ax.bar(dfs[i].index+counter, dfs[i][cols[i]], label = labels[i], width=1)
        # counter+=max(dfs[i].index)
    ax.set_xlabel(xlabel, fontsize=18)
    ax.set_ylabel(ylabel, fontsize=16)
    ax.tick_params(axis='x', labelsize=20)
    ax.tick_params(axis='y', labelsize=20)
    ax.legend(fontsize=12)
    ax.set_title(title, fontsize = 12)

In [ ]:
def main_func(df, user_params):
    #scale_input_vars = 'yes' if user_params['scale_input_vars'] else 'no'        
    art_path = Path(user_params['output_dir'], f'art__scaled_y_{scaling_factor}_no_scaled_inputs_run{run}.pkl')
    draws_path = Path(user_params['output_dir'], f'draws_scaled_x_{scaling_factor}_no_scaled_inputs_run{run}.pkl')
    print(f'run {run}, scaling factor {scaling_factor}')
    df_tmp = df.copy()
    df_tmp.loc[df_tmp["Measurement Type"] == user_params['response'], "Value"] = df_tmp.loc[df_tmp["Measurement Type"] == user_params['response'], "Value"]*scaling_factor
    df_tmp['Line Name']= df_tmp['Line Name'].apply(str)

    art_tmp = train_art(df_tmp, user_params, f'_scaled_x_{scaling_factor}_{scale_input_vars}_scaled_inputs_run{run}')
    art_tmp = generate_recommendations(art_tmp, user_params, f'_draws_scaled_x_{scaling_factor}_{scale_input_vars}_scaled_inputs_run{run}')
    return art_tmp

In [ ]:
user_params

In [ ]:

# outliers = ['C2.1_WA7_B1-R1','C2.1_WB8_C2-R1']

# df_stacked = df_stacked[~df_stacked['Line Name'].isin(outliers)]
# df_stacked[df_stacked['Measurement Type'] == 'OD340'].tail(48)

In [ ]:
art = train_art(df_stacked, user_params, 'art_yield_DBTL3.cpkl')
# import cloudpickle

# with open('../flaviolin data/DBTL3.2/art_x100_outlier_removed.cpkl', "rb") as f:
#     art = cloudpickle.load(f)


In [ ]:
# art.recommendations = pd.read_csv('../flaviolin data/DBTL3/recommendations_x100.csv')[user_params['components']].values
# recs_df = pd.read_csv('../flaviolin data/DBTL3/recommendations_x100.csv')
# recs_df.drop(columns=recs_df.columns[0], axis=1, inplace=True)
# art.post_pred_stats(recs_df[user_params['components']])

In [ ]:
# art.recommendations = pd.read_csv('../flaviolin data/DBTL3/recommendations_x100.csv')[user_params['components']]
# art.recommendations

In [ ]:
from scipy.optimize import differential_evolution, Bounds
def recommend_DE(art, user_params):
    def obj_func(x, art, draws):
        #print(x.shape)
        foo = art.post_pred_stats(x)
        #file1.write(str(x.flatten()[1:-1]))
        G = np.squeeze((1-art.alpha)*foo[0] +  art.alpha*foo[1])
        draws.append(np.append(x,G))

        return -G

    f = lambda x: obj_func(x,art, draws)
    draws = []
    result = differential_evolution(f, Bounds(art.bounds.values[:,0], art.bounds.values[:,1]), \
                                    maxiter = user_params['maxiter'], popsize= user_params['popsize'], vectorized = False,
                                    strategy = 'rand2exp', mutation =user_params['mutation'], recombination=user_params['recombination'], updating='deferred')

    draws = np.array(draws) #make into a numpy array
    draws_padded = np.hstack((draws, np.reshape(draws[:,-1], [-1,1]))) # add an extra column in the end to achieve n_input_vars+2

    from art.core.recommender import Recommender

    recommender = Recommender(
                art.loader,
                art.create_bounds(),
                art._args,
                publish_recs=False,
                # ### Pass-through RE params  #######
                # self.input_vars,
                # self.num_recommendations,
                # self.output_dir,
                # self.rel_rec_distance,
                # self.response_vars,
                # self.testing,
                # self.verbose,
                # self.warning_callback,
            )
    recommendations = recommender.select(draws_padded)
    return recommendations, draws

In [ ]:
user_params['maxiter'] = 3
user_params['popsize'] = 5000
user_params['mutation'] = (1,1.9)
user_params['recombination'] = 0.5


In [ ]:
art.alpha = user_params['aplha_explore']
art.num_recommendations = user_params['n_instances_explore']

df_recs_explore, draws_explore = recommend_DE(art, user_params)

In [ ]:
art.alpha = user_params['aplha_exploit']
art.num_recommendations = user_params['n_instances_exploit']

df_recs_exploit, draws_exploit = recommend_DE(art, user_params)

In [ ]:
df_recs_explore = df_recs_explore[art._args.input_vars]
df_recs_exploit = df_recs_exploit[art._args.input_vars]


mean_explore, std_explore = art.post_pred_stats(df_recs_explore[art._args.input_vars])
df_recs_explore['OD34yield_pred'] = mean_explore
df_recs_explore['OD34yield_std_pred'] = mean_explore

mean_exploit, std_exploit = art.post_pred_stats(df_recs_exploit[art._args.input_vars])
df_recs_exploit['OD34yield_pred'] = mean_exploit
df_recs_exploit['OD34yield_std_pred'] = mean_exploit


df_recs_explore['Label'] = 'explore'
df_recs_exploit['Label'] = 'exploit'


In [ ]:
fig, ax = plt.subplots()
ax.bar(range(art.X.shape[0]),art.predict(art.X).flatten(), label = 'training data predictions')
ax.bar(range(art.X.shape[0], art.X.shape[0]+user_params['n_instances_exploit']), np.array(df_recs_exploit['OD34yield_pred']).flatten(), label = 'exploit recs predictions')
ax.bar(range(art.X.shape[0]+user_params['n_instances_exploit'],art.X.shape[0]+user_params['n_instances_exploit']+user_params['n_instances_explore'])\
        , np.array(df_recs_explore['OD34yield_pred']).flatten(), label = 'explore recs predictions')
ax.legend()
plt.show()
fig.savefig(user_params['output_dir']+'/rec_bar_graph_x100_outlier_removed.png')

In [ ]:
df_target = pd.concat([df_recs_explore, df_recs_exploit]).reset_index()


In [ ]:
df_target

In [ ]:
user_params['standard_media_file']

In [ ]:
df_stand = pd.read_csv(user_params['standard_media_file']).set_index("Component")
df_stand = df_stand.rename(columns = {"Concentration":'Concentration[mM]'} )
df_stand

In [ ]:
ub = 1.1
lb = 0.9
df_control = pd.DataFrame(columns=user_params['components'])

for component in user_params['components']:
    stand_conc = df_stand.loc[component]['Concentration[mM]']
    df_control.at['Control', component] = stand_conc*np.random.uniform(lb, ub)

df_control

In [ ]:
for component in user_params['components'][:-1]:
     assert(all(df_control.at['Control', component] != control_lines[component]))

In [ ]:
control_predicted_mean, control_predicted_std = art.post_pred_stats(
            df_control.values
        )
df_control['OD340yield_pred'] = control_predicted_mean
df_control['OD340yield_std'] = control_predicted_std
df_control['Label'] = 'standard'

In [ ]:
df_control

In [ ]:
df_train = df[user_params['components']]
df_train['OD340'] = df['OD340']
df_train

In [ ]:
train_predicted_mean, train_predicted_std = art.post_pred_stats(
            df_train.values[:, :-1])
        
df_train['OD340yield_pred'] = train_predicted_mean
df_train['OD340yield_std'] = train_predicted_std
df_train['OD340yield_cv_pred'] = art.model_df_cv[0]["Predictions"]["Ensemble Model"]
df_train['OD340yield_cv_std'] = art.model_df_cv[0]["Predictions StDev"]["Ensemble Model"]
df_train['Label'] = 'train'

control_lines = df[df['Line Name'].str.find('_F8') > 0]

df_train.loc[control_lines.index, 'Label'] = 'standard'

In [ ]:
df_train

In [ ]:
file = f"{user_params['output_dir']}/train_pred.csv"
df_train.to_csv(file)


In [ ]:
df_target = pd.concat([df_target,df_control]).reset_index(drop=True)
df_target = df_target.loc[df_target.index.repeat(user_params['n_replicates'])]
df_target

In [ ]:
well_rows = 'ABCDEF'
well_columns = '12345678'
well_names = [f'{row}{column}'  for column in well_columns for row in well_rows]

df_target['Well'] = well_names
df_target = df_target.set_index(['Well'])
df_target.head()

In [ ]:
file = f"{user_params['output_dir']}/target_concentrations.csv"
df_target.to_csv(file) 